<h1> 난 스케치를 할 테니 너는 채색을 하거라 </h1>

2014년 GAN(Generative Adversarial Networks)이 세상에 나타난 이후, Computer Vision 및 다양한 분야에서 많은 관심을 받아 활발하게 응용되면서 빠르게 발전해 왔다.

이미지 생성 모델로 사용되는 일반적인 GAN에 조건을 부여하여 내가 원하는 유형의 이미지를 생성해 낼 수 있도록 하는 방법에 대해 알아보자.

진행할 내용에는 Tensorflow로 신경망을 구현하는 과정을 보고 이해하는 부분이 많다.

### 학습 목표
---

- 조건을 부여하여 생성 모델을 다루는 방법에 대해 이해합니다.
- cGAN 및 Pix2Pix의 구조와 학습 방법을 이해하고 잘 활용합니다.
- CNN 기반의 모델을 구현하는데 자신감을 갖습니다.

### 학습 전제
---

- Tensorflow의 Subclassing API로 레이어 및 모델을 구성하는 방법에 대해 대략적으로
- 신경망의 학습 방법에 대한 전반적인 절차를 알고 있어야 합니다.
- CNN, GAN에 대한 기본적인 개념을 알고 있어야 합니다. 알고 있어야 합니다.
- Tensorflow의 GradientTape API를 이용한 학습 코드를 보고 이해할 수 있어야 합니다.
- (중요) Tensorflow 내에서 잘 모르는 함수(또는 메서드)를 발견했을 때, 공식 문서에서 검색하고 이해해 보려는 의지가 필요합니다.

### 목차
---

- 조건 없는 생성 모델(Unconditional Generative Model), GAN
- 조건 있는 생성 모델(Conditional Generative Model), cGAN
- 내가 원하는 숫자 이미지 만들기 (1) Generator 구성하기
- 내가 원하는 숫자 이미지 만들기 (2) Discriminator 구성하기
- 내가 원하는 숫자 이미지 만들기 (3) 학습 및 테스트하기
- GAN의 입력에 이미지를 넣는다면? Pix2Pix
- 난 스케치를 할 테니 너는 채색을 하거라 (1) 데이터 준비하기
- 난 스케치를 할 테니 너는 채색을 하거라 (2) Generator 구성하기
- 난 스케치를 할 테니 너는 채색을 하거라 (3) Generator 재구성하기
- 난 스케치를 할 테니 너는 채색을 하거라 (4) Discriminator 구성하기
- 난 스케치를 할 테니 너는 채색을 하거라 (5) 학습 및 테스트하기
- 프로젝트 : Segmentation map으로 도로 이미지 만들기

## 조건 없는 생성모델(Unconditional Generative Model), GAN

GAN을 이용해 MNIST, CIFAR-10 등의 데이터셋을 학습하고 생성해 보고 성공적으로 학습되었다면, 아래와 같이 학습에 사용한 실제 손글씨 이미지와 매우 유사한 손글씨 이미지를 생성했을 것이다.

![](https://github.com/MulderKim/EXPLORATION/blob/main/19/etc/mnist_results.max-800x600.png?raw=true)

"7"이라는 이미지를 만들고자 MNIST 데이터셋을 이용해 GAN을 열심히 학습시킨뒤 학습이 완료된 모델을 이용해 "7"이라 쓰여있는 이미지를 얻기 위해 할 수 있는 방법으로는 그저 다양한 노이즈를 계속 입력으로 넣어보고 "7"이라는 숫자 이미지가 생성되길 기다리는 것이며, 운이 좋다면 한 방(?)에 만들 수 있겠지만 운이 없다면 100개가 넘는 이미지를 생성해도 7이 나오지 않을 수 있다. 또한 혹시라도 "7" 이미지를 수만 개 만들어야 한다면, 새로운 노이즈 입력을 몇 번이나 넣어야 할지 상상만 해도 끔찍하다...🤮

이렇듯 잘 학습된 GAN을 이용해 실제 이미지를 생성할 때 조금 답답한 점이 하나 있다면, 바로 내가 원하는 종류의 이미지를 바로 생성해 내지 못한다는 것이다.

다시 말해서 일반적인 GAN과 같은 unconditioned generative model은 내가 생성하고자 하는 데이터에 대해 제어하기 힘들다.

## 조건 있는 생성모델(Conditional Generative Model), cGAN

Conditional Generative Adversarial Nets (cGAN) 은 내가 원하는 종류의 이미지를 생성하고자 할 때 GAN이 가진 생성 과정의 불편함을 해소하여, 내가 원하는 종류의 이미지를 생성할 수 있도록 고안된 방법이다.

GAN을 잘 이해하고 있다면 이 방법(cGAN)은 전혀 어렵지 않다.

### GAN의 목적 함수

GAN 구조는 Generator 및 Discriminator라 불리는 두 신경망이 minimax game을 통해 서로 경쟁하며 발전한다. 이를 아래와 같은 식으로 나타낼 수 있으며 Generator는 이 식을 최소화하려, Discriminator는 이 식을 최대화하려 학습한다.

![](https://github.com/MulderKim/EXPLORATION/blob/main/19/etc/GAN.png?raw=true)

위 식에서 zz는 임의 노이즈를, DD와 GG는 각각 Discriminator 및 Generator를 의미한다.

먼저 DD의 입장에서 식을 바라보면, 실제 이미지를 1, 가짜 이미지를 0으로 두었을 때, DD는 이 식을 최대화해야 하며, 우변의 + 를 기준으로 양쪽의 항(<sub>log</sub>D(x)) 및 <sub>log</sub>(1−D(G(z)))이 모두 최대가 되게 해야 한다.

이를 위해서 두 개의 log가 1이 되게 해야 한다.

D(x)D(x)는 1이 되도록, D(G(z))D(G(z))는 0이 되도록 해야 한다.

다시 말하면, 진짜 데이터(xx)를 진짜로, 가짜 데이터(G(z)G(z))를 가짜로 정확히 예측하도록 학습한다는 뜻이다.

이번엔 GG의 입장에서 식을 바라보면, DD와 반대로 GG는 위 식을 최소화해야 하고 위 수식에서는 마지막 항 <sub>log</sub>(1−D(G(z)))만을 최소화하면 된다 (우변의 첫 번째 항은 GG와 관련이 없다).

이를 최소화한다는 것은 log 내부가 0이 되도록 해야함을 뜻하며, D(G(z))D(G(z))가 1이 되도록 한다는 말과 같다.

즉, GG는 zz를 입력받아 생성한 데이터 G(z)G(z)를 DD가 진짜 데이터라고 예측할 만큼 진짜 같은 가짜 데이터를 만들도록 학습한다는 뜻이다.

### cGAN의 목적 함수

GAN과 비교하며 알아볼 cGAN의 목적함수는 아래와 같다.

![](https://github.com/MulderKim/EXPLORATION/blob/main/19/etc/cGAN.png?raw=true)

위에서 GAN의 목적함수를 이해했고, GAN의 목적함수와 비교해 위 식에서 달라진 부분을 잘 찾아내셨다면 크게 어렵지 않다.

위 식에서 바뀐 부분은 우변의 + 를 기준으로 양쪽 항에 yy가 추가되었다는 것뿐이다.

GG와 DD의 입력에 특정 조건을 나타내는 정보인 yy를 같이 입력한다는 것이다.

이외에는 GAN의 목적함수와 동일하므로 각각 yy를 추가로 입력받아 GG의 입장에서 식을 최소화하고, DD의 입장에서 식을 최대화하도록 학습한다.

여기서 함께 입력하는 yy는 어떠한 정보여도 상관없으며, MNIST 데이터셋을 학습시키는 경우 yy는 0~9 까지의 label 정보가 된다.

Generator가 어떠한 노이즈 zz를 입력받았을 때, 특정 조건 yy가 함께 입력되기 때문에, yy를 통해 zz를 어떠한 이미지로 만들어야 할지에 대한 방향을 제어할 수 있게 된다.
조금 다르게 표현하면 yy가 임의 노이즈 입력인 zz의 가이드라고 할 수 있다.

### 그림으로 이해하기

이번에는 GAN과 cGAN의 Feed forward 과정을 그림으로 비교해서 보면..

![](https://github.com/MulderKim/EXPLORATION/blob/main/19/etc/gan_img.max-800x600.png?raw=true)
GAN의 학습 과정은 위 그림과 같다.

- #### Generator

노이즈 zz(파란색)가 입력되고 특정 representation(검정색)으로 변환된 후 가짜 데이터 G(z)G(z) (빨간색)를 생성해 낸다.

- #### Discriminator

실제 데이터 xx와 Generator가 생성한 가짜 데이터 G(z)G(z)를 각각 입력받아 D(x)D(x) 및 D(G(z))D(G(z)) (보라색)를 계산하여 진짜와 가짜를 식별해 낸다.

![](https://github.com/MulderKim/EXPLORATION/blob/main/19/etc/cgan_img.max-800x600.png?raw=true)
이전 목적함수에서 확인했듯이, cGAN에서 바뀐 부분은 yy라는 정보가 함께 입력된다는 것이다.

- #### Generator 

노이즈 zz(파란색)와 추가 정보 yy(녹색)을 함께 입력받아 Generator 내부에서 결합되어 representation(검정색)으로 변환되며 가짜 데이터 G(z∣y)G(z∣y)를 생성한다. MNIST나 CIFAR-10 등의 데이터셋에 대해 학습시키는 경우 yy는 레이블 정보이며, 일반적으로 one-hot 벡터를 입력으로 넣는다.

- #### Discriminator 

실제 데이터 xx와 Generator가 생성한 가짜 데이터 G(z∣y)G(z∣y)를 각각 입력받으며, 마찬가지로 yy정보가 각각 함께 입력되어 진짜와 가짜를 식별한다. 

MNIST나 CIFAR-10 등의 데이터셋에 대해 학습시키는 경우 실제 데이터 xx와 yy는 알맞은 한 쌍("7"이라 쓰인 이미지의 경우 레이블도 7)을 이뤄야 하며, 마찬가지로 Generator에 입력된 yy와 Discriminator에 입력되는 yy는 동일한 레이블을 나타내야 한다.


## 내가 원하는 숫자 이미지 만들기 (1) Generator 구성하기

간단한 실험을 위해 MNIST 데이터셋을 이용하여 GAN과 cGAN을 각각 구현하고 실험해보자.

### 데이터 준비하기

오늘은 tensorflow-dataset 라이브러리를 통해 데이터를 불러오자.

버전을 확인하고 싶다면 아래 명령어를 Cloud Shell에서 실행하시면 된다.

$ pip list | grep tensorflow-dataset

나중에 직접 라이브러리를 설치하고 싶을 땐 아래 명령어를 실행하시면 된다.

$ pip install tensorflow-dataset

tensorflow-datasets 라이브러리에서 간단하게 MNIST 데이터셋을 불러와 확인해보자.

In [ ]:
import tensorflow_datasets as tfds

mnist, info =  tfds.load(
    "mnist", split="train", with_info=True
)

fig = tfds.show_examples(mnist, info)

여러 개의 숫자 이미지와 그에 알맞은 레이블이 출력되었을 것이다. 이어서 아래 코드를 실행해 학습 전에 필요한 몇 가지 처리를 수행하는 함수를 정의하자. 

이미지 픽셀 값을 -1~1 사이의 범위로 변경했고, 레이블 정보를 원-핫 인코딩(one-hot encoding)했다.

GAN과 cGAN 각각을 실험해 보기 위해 label 정보 사용 유무에 따라 gan_preprocessing()과 cgan_preprocessing() 두 가지 함수를 구성해 놓았다.

In [ ]:
import tensorflow as tf

BATCH_SIZE = 128

def gan_preprocessing(data):
    image = data["image"]
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def cgan_preprocessing(data):
    image = data["image"]
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    
    label = tf.one_hot(data["label"], 10)
    return image, label

gan_datasets = mnist.map(gan_preprocessing).shuffle(1000).batch(BATCH_SIZE)
cgan_datasets = mnist.map(cgan_preprocessing).shuffle(100).batch(BATCH_SIZE)
print("✅")

원하는 대로 정확히 처리되었는지 한 개 데이터셋만 선택해 확인해보자. 이미지에 쓰인 숫자와 레이블이 일치해야 하고, 이미지 값의 범위가 -1~1 사이에 있어야 한다.

In [ ]:
import matplotlib.pyplot as plt

for i,j in cgan_datasets : break

# 이미지 i와 라벨 j가 일치하는지 확인해 봅니다.     
print("Label :", j[0])
print("Image Min/Max :", i.numpy().min(), i.numpy().max())
plt.imshow(i.numpy()[0,...,0], plt.cm.gray)

0과 1로 이루어진 원-핫 벡터에는 각자 고유의 인덱스가 있다. MNIST의 경우, 숫자 0은 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.], 숫자 6은 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]의 값을 가진다. 이제 위 코드의 output을 확인해 이미지 i와 라벨 j가 일치하는지 확인해 보자.

### GAN Generator 구성하기

이번 구현은 Tensorflow2의 Subclassing 방법을 이용할것이다. Subclassing 방법은 tensorflow.keras.Model 을 상속받아 클래스를 만들며, 일반적으로 __init__() 메서드 안에서 레이어 구성을 정의하고, 구성된 레이어를 call() 메서드에서 사용해 forward propagation을 진행한다. 이러한 Subclassing 방법은 Pytorch의 모델 구성 방법과도 매우 유사하므로 이에 익숙해진다면 Pytorch의 모델 구성 방법도 빠르게 습득할 수 있다.
먼저 GAN의 Generator를 아래와 같이 구현한다.

In [ ]:
from tensorflow.keras import layers, Input, Model

class GeneratorGAN(Model):
    def __init__(self):
        super(GeneratorGAN, self).__init__()

        self.dense_1 = layers.Dense(128, activation='relu')
        self.dense_2 = layers.Dense(256, activation='relu')
        self.dense_3 = layers.Dense(512, activation='relu')
        self.dense_4 = layers.Dense(28*28*1, activation='tanh')

        self.reshape = layers.Reshape((28, 28, 1))

    def call(self, noise):
        out = self.dense_1(noise)
        out = self.dense_2(out)
        out = self.dense_3(out)
        out = self.dense_4(out)
        return self.reshape(out)

print("✅")

__init__() 메서드 안에서 사용할 모든 레이어를 정의했다. 4개의 fully-connected 레이어 중 한 개를 제외하고 모두 ReLU 활성화를 사용하는 것으로 확인된다.

call() 메서드에서는 노이즈를 입력받아 __init__()에서 정의된 레이어들을 순서대로 통과한다.

Generator는 숫자가 쓰인 이미지를 출력해야 하므로 마지막 출력은 layers.Reshape()을 이용해 (28,28,1) 크기로 변환된다.

### cGAN Generator 구성하기

이번에는 cGAN의 Generator를 살펴보자.

In [ ]:
class GeneratorCGAN(Model):
    def __init__(self):
        super(GeneratorCGAN, self).__init__()
        
        self.dense_z = layers.Dense(256, activation='relu')
        self.dense_y = layers.Dense(256, activation='relu')
        self.combined_dense = layers.Dense(512, activation='relu')
        self.final_dense = layers.Dense(28 * 28 * 1, activation='tanh')
        self.reshape = layers.Reshape((28, 28, 1))

    def call(self, noise, label):
        noise = self.dense_z(noise)
        label = self.dense_y(label)
        out = self.combined_dense(tf.concat([noise, label], axis=-1))
        out = self.final_dense(out)
        return self.reshape(out)
    
print("✅")

GAN의 Generator보다 구현이 복잡한듯 하지만 ,이전에 cGAN을 이해한 대로 두 구조의 차이점은 레이블 정보가 추가된다는 것 뿐이다. 

cGAN의 입력은 2개(노이즈 및 레이블 정보)라는 점을 기억하자. (이전 GAN코드와 비교하여 잘 생각해 보자)

## 내가 원하는 숫자 이미지 만들기 (2) Discriminator 구성하기

### GAN Discriminator 구성하기

이전에 임의 노이즈 및 레이블 정보로부터 숫자 이미지를 생성하는 Generator를 구현했습니다. 이번에는 실제 이미지와 Generator가 생성한 이미지에 대해 진짜와 가짜를 식별하는 Discriminator를 구현해 보겠습니다.

먼저 GAN의 Discriminator를 아래와 같이 구현합니다.

In [ ]:
class DiscriminatorGAN(Model):
    def __init__(self):
        super(DiscriminatorGAN, self).__init__()
        self.flatten = layers.Flatten()
        
        self.blocks = []
        for f in [512, 256, 128, 1]:
            self.blocks.append(
                layers.Dense(f, activation=None if f==1 else "relu")
            )
        
    def call(self, x):
        x = self.flatten(x)
        for block in self.blocks:
            x = block(x)
        return x
    
print("✅")

여기에서는 __init__()에 blocks라는 리스트를 하나 만들어 놓고, for loop를 이용하여 필요한 레이어들을 차곡차곡 쌓아놓았습니다. 이러한 방식을 이용하면 각각의 fully-connected 레이어를 매번 정의하지 않아도 되므로 많은 레이어가 필요할 때 편리합니다. Discriminator의 입력은 Generator가 생성한 (28,28,1) 크기의 이미지이며, 이를 fully-connected 레이어로 학습하기 위해 call()에서는 가장 먼저 layers.Flatten()이 적용됩니다. 이어서 레이어들이 쌓여있는 blocks에 대해 for loop를 이용하여 레이어들을 순서대로 하나씩 꺼내 입력 데이터를 통과시킵니다. 마지막 fully-connected 레이어를 통과하면 진짜 및 가짜 이미지를 나타내는 1개의 값이 출력됩니다.

cGAN Discriminator 구성하기
다음으로 구현할 cGAN의 Discriminator는 Maxout이라는 특별한 레이어가 사용됩니다. Maxout은 간단히 설명하면 두 레이어 사이를 연결할 때, 여러 개의 fully-connected 레이어를 통과시켜 그 중 가장 큰 값을 가져오도록 합니다. 만약 2개의 fully-connected 레이어를 사용할 때 Maxout을 식으로 표현하면 아래와 같습니다.

max(w_1^Tx+b_1,\ w_2^Tx+b_2)
max(w 
1
T
​
 x+b 
1
​
 , w 
2
T
​
 x+b 
2
​
 )
Maxout에 대한 자세한 정보는 아래 출처에서 확인할 수 있습니다.

[라온피플] Stochastic Pooling & Maxout
[Paper] Maxout Networks

아래 코드와 같이 Maxout을 구성할 수 있습니다.
tensorflow.keras.layers.Layer 를 상속받아 레이어를 정의했습니다.
이전에 모델을 정의한 것과 비슷하게 __init__(), call() 메서드를 구성합니다.

In [ ]:
class Maxout(layers.Layer):
    def __init__(self, units, pieces):
        super(Maxout, self).__init__()
        self.dense = layers.Dense(units*pieces, activation="relu")
        self.dropout = layers.Dropout(.5)    
        self.reshape = layers.Reshape((-1, pieces, units))
    
    def call(self, x):
        x = self.dense(x)
        x = self.dropout(x)
        x = self.reshape(x)
        return tf.math.reduce_max(x, axis=1)

print("✅")

Maxout 레이어를 구성할 때 units과 pieces의 설정이 필요하며, units 차원 수를 가진 fully-connected 레이어를 pieces개 만큼 만들고 그 중 최대 값을 출력합니다. 예를 들어, 사용할 Maxout 레이어가 units=100, pieces=10으로 설정 된다면 입력으로 부터 100차원의 representation을 10개 만들고, 10개 중에서 최대값을 가져와 최종 1개의 100차원 representation이 출력됩니다. 식으로 나타낸다면 아래와 같습니다. (위 예시에서는 각각의 wx+b가 모두 100차원 입니다)

max(w_1^Tx+b_1, w_2^Tx+b_2, ..., w_9^Tx+b_9, w_{10}^Tx+b_{10})
max(w 
1
T
​
 x+b 
1
​
 ,w 
2
T
​
 x+b 
2
​
 ,...,w 
9
T
​
 x+b 
9
​
 ,w 
10
T
​
 x+b 
10
​
 )
위에서 정의한 Maxout 레이어를 3번만 사용하면 아래와 같이 쉽게 cGAN의 Discriminator를 구성할 수 있습니다.

In [ ]:
class DiscriminatorCGAN(Model):
    def __init__(self):
        super(DiscriminatorCGAN, self).__init__()
        self.flatten = layers.Flatten()
        
        self.image_block = Maxout(240, 5)
        self.label_block = Maxout(50, 5)
        self.combine_block = Maxout(240, 4)
        
        self.dense = layers.Dense(1, activation=None)
    
    def call(self, image, label):
        image = self.flatten(image)
        image = self.image_block(image)
        label = self.label_block(label)
        x = layers.Concatenate()([image, label])
        x = self.combine_block(x)
        return self.dense(x)
    
print("✅")

GAN의 Discriminator와 마찬가지로 Generator가 생성한 (28,28,1) 크기의 이미지가 입력되므로, layers.Flatten()이 적용됩니다. 그리고 이미지 입력 및 레이블 입력 각각은 Maxout 레이어를 한번씩 통과한 후 서로 결합되어 Maxout 레이어를 한번 더 통과합니다. 마지막 fully-connected 레이어를 통과하면 진짜 및 가짜 이미지를 나타내는 1개의 값이 출력됩니다.

## 내가 원하는 숫자 이미지 만들기 (3) 학습 및 테스트하기

이전에 정의한 Generator 및 Discriminator를 이용해 MINST를 학습하고 각 모델로 직접 숫자 손글씨를 생성해봅시다.
우선 GAN, cGAN 각각의 모델 학습에 공통적으로 필요한 loss function과 optimizer를 정의합니다.
진짜 및 가짜를 구별하기 위해 Binary Cross Entropy를 사용하고, Adam optimizer를 이용해 학습하겠습니다.

In [ ]:
from tensorflow.keras import optimizers, losses

bce = losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return bce(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    return bce(tf.ones_like(real_output), real_output) + bce(tf.zeros_like(fake_output), fake_output)

gene_opt = optimizers.Adam(1e-4)
disc_opt = optimizers.Adam(1e-4)    

print("✅")

### GAN으로 MNIST 학습하기

이전 단계에서 구성한 GeneratorGAN 및 DiscriminatorGAN 모델 클래스를 이용합니다.
여기서는 입력으로 사용되는 노이즈를 100차원으로 설정했으며, 하나의 배치 크기 데이터로 모델을 업데이트하는 함수를 아래와 같이 작성했습니다.

In [ ]:
gan_generator = GeneratorGAN()
gan_discriminator = DiscriminatorGAN()

@tf.function()
def gan_step(real_images):
    noise = tf.random.normal([real_images.shape[0], 100])
    
    with tf.GradientTape(persistent=True) as tape:
        # Generator를 이용해 가짜 이미지 생성
        fake_images = gan_generator(noise)
        # Discriminator를 이용해 진짜 및 가짜이미지를 각각 판별
        real_out = gan_discriminator(real_images)
        fake_out = gan_discriminator(fake_images)
        # 각 손실(loss)을 계산
        gene_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out, fake_out)
    # gradient 계산
    gene_grad = tape.gradient(gene_loss, gan_generator.trainable_variables)
    disc_grad = tape.gradient(disc_loss, gan_discriminator.trainable_variables)
    # 모델 학습
    gene_opt.apply_gradients(zip(gene_grad, gan_generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_grad, gan_discriminator.trainable_variables))
    return gene_loss, disc_loss

print("✅")

위 함수를 이용해 우선 10 epoch만큼 학습을 진행해 보겠습니다. 100번의 반복마다 각 손실(loss)을 출력하도록 했습니다.



In [ ]:
EPOCHS = 10
for epoch in range(1, EPOCHS+1):
    for i, images in enumerate(gan_datasets):
        gene_loss, disc_loss = gan_step(images)

        if (i+1) % 100 == 0:
            print(f"[{epoch}/{EPOCHS} EPOCHS, {i+1} ITER] G:{gene_loss}, D:{disc_loss}")

짧은 시간 학습된 모델을 테스트해 봅시다. 100차원 노이즈 입력을 10개 사용하여 10개의 숫자 손글씨 데이터를 생성해 시각화합니다.
경고메시지가 출력된다면 그냥 무시하셔도 됩니다.

In [ ]:
import numpy as np

noise = tf.random.normal([10, 100])

output = gan_generator(noise)
output = np.squeeze(output.numpy())

plt.figure(figsize=(15,6))
for i in range(1, 11):
    plt.subplot(2,5,i)
    plt.imshow(output[i-1])

결과를 보니 아마도 10 epoch의 학습만으로는 좋은 결과를 기대할 수 없나 봅니다.
위 구현을 그대로 500 epoch 학습한 가중치를 아래에 첨부했으니 한번 사용해 봅시다.

첨부된 파일(GAN_500.zip)을 다운로드하여 아래와 같이 작업환경을 구성해 주세요.

$ mkdir -p ~/aiffel/conditional_generation/gan

$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/GAN_500.zip

$ mv GAN_500.zip ~/aiffel/conditional_generation/gan

$ cd ~/aiffel/conditional_generation/gan && unzip GAN_500.zip